In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score
import optuna
from tqdm.notebook import tqdm
import random

pd.options.display.max_columns = 100
pd.options.display.max_rows = 50
optuna.logging.set_verbosity(0)

seed = 777

random.seed(seed)
np.random.seed(seed)

In [2]:
train = pd.read_csv('data/train.csv').drop('ID_y', axis=1)
sample = pd.read_csv('data/sample_solution.csv')
test = pd.read_csv('data/test_dataset_test.csv')

train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

test.loc[test['Статус Курения']=='Никогда не курил', 'Статус Курения'] = 'Никогда не курил(а)'
test.loc[test['Религия'].isin(['Другое', 'Индуизм']), 'Религия'] = 'Нет'

In [3]:
def get_features(df):
    df['Время засыпания'] = pd.to_datetime(df['Время засыпания']).dt.hour + pd.to_datetime(df['Время засыпания']).dt.minute / 60
    df.loc[df['Время засыпания']<12, 'Время засыпания'] += 24
    df['Время пробуждения'] = pd.to_datetime(df['Время пробуждения']).dt.hour + pd.to_datetime(df['Время пробуждения']).dt.minute / 60
    df.loc[df['Время пробуждения']<12, 'Время пробуждения'] += 24
    
    for x in ['Сигарет в день', 'Возраст курения', 'Возраст алког']:
        df[x] = df[x].fillna(0)
        
    df.loc[df['Национальность']!='Русские', 'Национальность'] = 0
    df.loc[df['Национальность']=='Русские', 'Национальность'] = 1
    
get_features(train)
get_features(test)

In [4]:
all_dicts = {}

cols = ['Пол', 'Семья', 'Этнос', 'Образование', 'Частота пасс кур', 'Алкоголь', 'Статус Курения', 'Религия', 'Профессия']

for x in cols:
    all_dicts[x] = LabelEncoder().fit(train[x])
    train[x] = all_dicts[x].transform(train[x])
    test[x] = all_dicts[x].transform(test[x])

In [5]:
target_cols = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']

features = train.drop(target_cols, axis=1)
target = train[target_cols].values

In [6]:
cat_f = ['Пол', 'Семья', 'Этнос', 'Образование', 'Вы работаете?',
       'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет',
       'Гепатит', 'Онкология', 'Хроническое заболевание легких',
       'Бронжиальная астма', 'Туберкулез легких ',
       'Регулярный прим лекарственных средств', 'Переломы',
       'Статус Курения', 'Частота пасс кур', 'Алкоголь', 'Сон после обеда',
       'Пассивное курение', 'Спорт, клубы', 'Религия, клубы',
       'Профессия', 'Религия']

In [8]:
def get_weights(i):
    if i == 0: return [0.49, 0.51]
    elif i == 1: return [0.15, 0.85]
    elif i == 2: return [0.45, 0.65]
    elif i == 3: return [0.3, 0.7]
    elif i == 4: return [0.3, 0.7]
    

def objective(trial):
    borders = trial.suggest_float('threshold', 0.001, 0.999)
    score = recall_score(y_valid, (valid_predictions > borders).astype('int'), average='macro')
    return score

In [9]:
models = []
all_scores = []
for x in tqdm(range(5)):
    cur_target = target[:, x]

    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    test_predictions = np.zeros(len(test))
    all_valid_labels = []
    all_valid_preds = []

    weights = get_weights(x)

    for fold, (train_index, valid_index) in enumerate(strat_kfold.split(features, cur_target)):
        X_train, y_train = features.iloc[train_index], cur_target[train_index]
        X_valid, y_valid = features.iloc[valid_index], cur_target[valid_index]

        valid_predictions = np.zeros(len(X_valid))
        cur_test_preds = np.zeros(len(test))

        for j in range(3):
            model = CatBoostClassifier(
                depth=4,
                learning_rate=0.02,
                iterations=3000,
                random_seed=seed+j,
                eval_metric='AUC',
                early_stopping_rounds=2000,
                class_weights=weights,
                logging_level='Silent'
            )

            model.fit(Pool(X_train, y_train, cat_features=cat_f), eval_set=Pool(X_valid, y_valid, cat_features=cat_f), verbose=200)
            valid_predictions += model.predict_proba(X_valid)[:, 1]
            cur_test_preds += model.predict_proba(test)[:, 1]
        
        valid_predictions /= 5
        cur_test_preds /= 5

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=1000)
        threshold = study.best_params['threshold']

        # test_predictions += (cur_test_preds > threshold).astype('int')
        test_predictions += cur_test_preds - threshold
        
        all_valid_labels.append(y_valid)
        all_valid_preds.append((valid_predictions > threshold).astype('int'))
        
    all_scores.append(recall_score(np.concatenate(all_valid_labels), np.concatenate(all_valid_preds), average='macro'))
    sample[target_cols[x]] = (test_predictions > 0).astype('int')

np.mean(all_scores)

  0%|          | 0/5 [00:00<?, ?it/s]

0.722356788586265

In [12]:
sample.to_csv('submit_new.csv', index=False)